In [3]:
import os
import os.path as osp

import time
import os, mmap
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import add_self_loops
from sklearn.manifold import TSNE
import numpy as np
from torch_geometric.datasets import Planetoid
import nvsmi
import threading
import psutil

# Using MMAP LoadData

In [4]:
from ctypes import *
result = cdll.LoadLibrary("./mmaploader.so")


# str_test="need_bytes"  
# val=str_test.encode()  
# func.strTest.restype = c_char_p
# file = c_char("/mnt/mem/cora.cites")
start_time = time.perf_counter()

file = bytes("/mnt/mem/cora.cites",encoding='utf-8')
result.readMMap.restype = c_char_p
cites = result.readMMap(file, 0, 10000);

file = bytes("/mnt/mem/cora.content",encoding='utf-8')
result.readMMap.restype = c_char_p
content = result.readMMap(file, 0, 200000000);

end_time = time.perf_counter()
# result.closemmap(bytes(m,encoding='utf-8'))

In [5]:
print('Loading time: ' + str(end_time - start_time))

Loading time: 0.016189693997148424


In [6]:
print(cites.decode())
print(content.decode())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
# 索引字典，将原本的论文id转换到从0开始编码
index_dict = dict()
# 标签字典，将字符串标签转化为数值
label_to_index = dict()

features = []
labels = []
edge_index = []
# start timer
start_time = time.perf_counter()

nodes = content.decode().split("\n")[:-1]
for node in nodes:
    node_info = node.split()
    print(node_info)
    index_dict[int(node_info[0])] = len(index_dict)
    features.append([int(i) for i in node_info[1:-1]])

    label_str = node_info[-1]
    if (label_str not in label_to_index.keys()):
        label_to_index[label_str] = len(label_to_index)
    labels.append(label_to_index[label_str])

edges = cites.decode().split("\n")[:-1]
for edge in edges:
    start, end = edge.split()
    # 训练时将边视为无向的，但原本的边是有向的，因此需要正反添加两次
    edge_index.append([index_dict[int(start)], index_dict[int(end)]])
    edge_index.append([index_dict[int(end)], index_dict[int(start)]])

# end timer
end_time = time.perf_counter()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['102061', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

In [11]:
print('Data proccessing time: ' + str(end_time - start_time))

Data proccessing time: 1.3682684359955601


# Using DirectIO LoadData

In [12]:
import directio
path = "/mnt/mem/"
cites = path + "cora.cites"
content = path + "cora.content"

# 索引字典，将原本的论文id转换到从0开始编码
index_dict = dict()
# 标签字典，将字符串标签转化为数值
label_to_index = dict()

features = []
labels = []
edge_index = []
# start timer
start_time = time.perf_counter()

# fd = open(cites, "r")
fd_o = os.open(content, os.O_DIRECT | os.O_RDONLY)
# print(fd)
# m = mmap.mmap(fd, 0)
content = directio.read(fd_o,10521600)

fd_o1 = os.open(cites, os.O_DIRECT | os.O_RDONLY)
cites = directio.read(fd_o1,70144)

end_time = time.perf_counter()
print('Loading time: ' + str(end_time - start_time))



Loading time: 0.004837907996261492


In [13]:
start_time = time.perf_counter()
nodes = content.decode().split("\n")[:-1]
for node in nodes:
    node_info = node.split()
    print(node_info)
    index_dict[int(node_info[0])] = len(index_dict)
    features.append([int(i) for i in node_info[1:-1]])

    label_str = node_info[-1]
    if (label_str not in label_to_index.keys()):
        label_to_index[label_str] = len(label_to_index)
    labels.append(label_to_index[label_str])

edges = cites.decode().split("\n")[:-1]
for edge in edges:
    start, end = edge.split()
    # 训练时将边视为无向的，但原本的边是有向的，因此需要正反添加两次
    edge_index.append([index_dict[int(start)], index_dict[int(end)]])
    edge_index.append([index_dict[int(end)], index_dict[int(start)]])
end_time = time.perf_counter()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
print('Data proccessing time: ' + str(end_time - start_time))

Data proccessing time: 1.2981013760145288


In [15]:
# To Tensor
labels = torch.LongTensor(labels)
features = torch.FloatTensor(features)
# norm
# features = torch.nn.functional.normalize(features, p=1, dim=1)
edge_index = torch.LongTensor(edge_index)


In [16]:
class GCNNet(torch.nn.Module):
    def __init__(self, num_feature, num_label):
        super(GCNNet, self).__init__()
        self.GCN1 = GCNConv(num_feature, 16)
        self.GCN2 = GCNConv(16, num_label)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.GCN1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.GCN2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [17]:
class GATNet(torch.nn.Module):
    def __init__(self, num_feature, num_label):
        super(GATNet, self).__init__()
        self.GAT1 = GATConv(num_feature, 8, heads=8, concat=True, dropout=0.6)
        self.GAT2 = GATConv(8 * 8, num_label, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.GAT1(x, edge_index)
        x = F.relu(x)
        x = self.GAT2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [18]:
class SAGENet(torch.nn.Module):
    def __init__(self, num_feature, num_label):
        super(SAGENet, self).__init__()
        self.conv1 = SAGEConv(num_feature, 128)
        self.conv2 = SAGEConv(128, num_label)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

In [19]:
seed = 1234
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)  # Numpy module.
# random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

mask = torch.randperm(len(index_dict))
train_mask = mask[:140]
val_mask = mask[140:640]
test_mask = mask[1708:2708]

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

data = Data(x=features, edge_index=edge_index.t().contiguous(), y=labels).to(device)
print(data)
#model
model = GCNNet(features.shape[1], len(label_to_index)).to(device)

print(model)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


# for epoch in range(1):
#     optimizer.zero_grad()
#     out = model(cora)
#     loss = F.nll_loss(out[train_mask], cora.y[train_mask])
#     #print('epoch: %d loss: %.4f' % (epoch, loss))
#     loss.backward()
#     optimizer.step()

#     if ((epoch + 1) % 10 == 0):
#         model.eval()
#         _, pred = model(cora).max(dim=1)
#         correct = int(pred[test_mask].eq(cora.y[test_mask]).sum().item())
#         acc = correct / len(test_mask)
#         print('Accuracy: {:.4f}'.format(acc))
#         model.train()
        
model.train()
for epoch in range(1000):
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, pred = torch.max(out[train_mask], dim=1)
    correct = (pred == data.y[train_mask]).sum().item()
    acc = correct/len(train_mask)
    print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
        epoch, loss.item(), acc))
    
    if ((epoch + 1) % 10 == 0):
        model.eval()
        _, pred = model(data).max(dim=1)
        correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
        acc = correct / len(test_mask)
        print('Accuracy: {:.4f}'.format(acc))
        model.train()
    
# for epoch in range(200):
#     out = model(data)
#     loss = criterion(out[data.train_mask], data.y[data.train_mask])
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     _, pred = torch.max(out[data.train_mask], dim=1)
#     correct = (pred == data.y[data.train_mask]).sum().item()
#     acc = correct/data.train_mask.sum().item()

#     print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
#         epoch, loss.item(), acc))

Data(x=[2708, 1433], edge_index=[2, 10858], y=[2708])
GCNNet(
  (GCN1): GCNConv(1433, 16)
  (GCN2): GCNConv(16, 7)
  (dropout): Dropout(p=0.5, inplace=False)
)
Epoch 000 train_loss: 1.9465 train_acc: 0.1714
Epoch 001 train_loss: 1.8267 train_acc: 0.4214
Epoch 002 train_loss: 1.7008 train_acc: 0.5286
Epoch 003 train_loss: 1.5580 train_acc: 0.6429
Epoch 004 train_loss: 1.4211 train_acc: 0.7071
Epoch 005 train_loss: 1.2882 train_acc: 0.7214
Epoch 006 train_loss: 1.1365 train_acc: 0.7857
Epoch 007 train_loss: 1.0425 train_acc: 0.8357
Epoch 008 train_loss: 0.9241 train_acc: 0.8714
Epoch 009 train_loss: 0.8349 train_acc: 0.9000
Accuracy: 0.7420
Epoch 010 train_loss: 0.7883 train_acc: 0.9000
Epoch 011 train_loss: 0.6423 train_acc: 0.9071
Epoch 012 train_loss: 0.5955 train_acc: 0.8929
Epoch 013 train_loss: 0.5107 train_acc: 0.9571
Epoch 014 train_loss: 0.4093 train_acc: 0.9500
Epoch 015 train_loss: 0.3849 train_acc: 0.9571
Epoch 016 train_loss: 0.4078 train_acc: 0.9500
Epoch 017 train_loss: 0.

Accuracy: 0.8210
Epoch 180 train_loss: 0.0259 train_acc: 1.0000
Epoch 181 train_loss: 0.0312 train_acc: 1.0000
Epoch 182 train_loss: 0.0397 train_acc: 1.0000
Epoch 183 train_loss: 0.0319 train_acc: 0.9929
Epoch 184 train_loss: 0.0355 train_acc: 0.9929
Epoch 185 train_loss: 0.0271 train_acc: 1.0000
Epoch 186 train_loss: 0.0225 train_acc: 1.0000
Epoch 187 train_loss: 0.0399 train_acc: 0.9929
Epoch 188 train_loss: 0.0308 train_acc: 1.0000
Epoch 189 train_loss: 0.0320 train_acc: 1.0000
Accuracy: 0.8230
Epoch 190 train_loss: 0.0312 train_acc: 1.0000
Epoch 191 train_loss: 0.0225 train_acc: 1.0000
Epoch 192 train_loss: 0.0242 train_acc: 1.0000
Epoch 193 train_loss: 0.0320 train_acc: 1.0000
Epoch 194 train_loss: 0.0414 train_acc: 1.0000
Epoch 195 train_loss: 0.0365 train_acc: 0.9929
Epoch 196 train_loss: 0.0316 train_acc: 0.9929
Epoch 197 train_loss: 0.0419 train_acc: 0.9857
Epoch 198 train_loss: 0.0456 train_acc: 0.9929
Epoch 199 train_loss: 0.0349 train_acc: 1.0000
Accuracy: 0.8220
Epoch 200

Epoch 349 train_loss: 0.0281 train_acc: 1.0000
Accuracy: 0.8240
Epoch 350 train_loss: 0.0299 train_acc: 0.9929
Epoch 351 train_loss: 0.0299 train_acc: 0.9929
Epoch 352 train_loss: 0.0313 train_acc: 0.9929
Epoch 353 train_loss: 0.0183 train_acc: 1.0000
Epoch 354 train_loss: 0.0195 train_acc: 1.0000
Epoch 355 train_loss: 0.0186 train_acc: 1.0000
Epoch 356 train_loss: 0.0213 train_acc: 1.0000
Epoch 357 train_loss: 0.0289 train_acc: 1.0000
Epoch 358 train_loss: 0.0354 train_acc: 0.9929
Epoch 359 train_loss: 0.0237 train_acc: 1.0000
Accuracy: 0.8190
Epoch 360 train_loss: 0.0203 train_acc: 1.0000
Epoch 361 train_loss: 0.0145 train_acc: 1.0000
Epoch 362 train_loss: 0.0184 train_acc: 1.0000
Epoch 363 train_loss: 0.0258 train_acc: 1.0000
Epoch 364 train_loss: 0.0222 train_acc: 1.0000
Epoch 365 train_loss: 0.0191 train_acc: 1.0000
Epoch 366 train_loss: 0.0190 train_acc: 1.0000
Epoch 367 train_loss: 0.0143 train_acc: 1.0000
Epoch 368 train_loss: 0.0296 train_acc: 1.0000
Epoch 369 train_loss: 0.02

Epoch 527 train_loss: 0.0311 train_acc: 0.9929
Epoch 528 train_loss: 0.0180 train_acc: 1.0000
Epoch 529 train_loss: 0.0184 train_acc: 0.9929
Accuracy: 0.8320
Epoch 530 train_loss: 0.0147 train_acc: 1.0000
Epoch 531 train_loss: 0.0188 train_acc: 1.0000
Epoch 532 train_loss: 0.0102 train_acc: 1.0000
Epoch 533 train_loss: 0.0164 train_acc: 1.0000
Epoch 534 train_loss: 0.0143 train_acc: 1.0000
Epoch 535 train_loss: 0.0095 train_acc: 1.0000
Epoch 536 train_loss: 0.0283 train_acc: 0.9929
Epoch 537 train_loss: 0.0209 train_acc: 1.0000
Epoch 538 train_loss: 0.0219 train_acc: 1.0000
Epoch 539 train_loss: 0.0157 train_acc: 1.0000
Accuracy: 0.8270
Epoch 540 train_loss: 0.0181 train_acc: 1.0000
Epoch 541 train_loss: 0.0180 train_acc: 1.0000
Epoch 542 train_loss: 0.0169 train_acc: 0.9929
Epoch 543 train_loss: 0.0157 train_acc: 1.0000
Epoch 544 train_loss: 0.0163 train_acc: 1.0000
Epoch 545 train_loss: 0.0221 train_acc: 1.0000
Epoch 546 train_loss: 0.0078 train_acc: 1.0000
Epoch 547 train_loss: 0.01

Epoch 699 train_loss: 0.0154 train_acc: 1.0000
Accuracy: 0.8270
Epoch 700 train_loss: 0.0184 train_acc: 1.0000
Epoch 701 train_loss: 0.0100 train_acc: 1.0000
Epoch 702 train_loss: 0.0272 train_acc: 1.0000
Epoch 703 train_loss: 0.0150 train_acc: 1.0000
Epoch 704 train_loss: 0.0093 train_acc: 1.0000
Epoch 705 train_loss: 0.0191 train_acc: 1.0000
Epoch 706 train_loss: 0.0196 train_acc: 1.0000
Epoch 707 train_loss: 0.0115 train_acc: 1.0000
Epoch 708 train_loss: 0.0246 train_acc: 0.9929
Epoch 709 train_loss: 0.0182 train_acc: 1.0000
Accuracy: 0.8320
Epoch 710 train_loss: 0.0216 train_acc: 0.9929
Epoch 711 train_loss: 0.0155 train_acc: 0.9929
Epoch 712 train_loss: 0.0256 train_acc: 1.0000
Epoch 713 train_loss: 0.0210 train_acc: 0.9929
Epoch 714 train_loss: 0.0160 train_acc: 1.0000
Epoch 715 train_loss: 0.0303 train_acc: 0.9786
Epoch 716 train_loss: 0.0194 train_acc: 1.0000
Epoch 717 train_loss: 0.0152 train_acc: 1.0000
Epoch 718 train_loss: 0.0206 train_acc: 1.0000
Epoch 719 train_loss: 0.02

Epoch 884 train_loss: 0.0197 train_acc: 1.0000
Epoch 885 train_loss: 0.0174 train_acc: 1.0000
Epoch 886 train_loss: 0.0132 train_acc: 1.0000
Epoch 887 train_loss: 0.0275 train_acc: 0.9929
Epoch 888 train_loss: 0.0122 train_acc: 1.0000
Epoch 889 train_loss: 0.0080 train_acc: 1.0000
Accuracy: 0.8290
Epoch 890 train_loss: 0.0186 train_acc: 0.9929
Epoch 891 train_loss: 0.0265 train_acc: 0.9929
Epoch 892 train_loss: 0.0275 train_acc: 0.9929
Epoch 893 train_loss: 0.0227 train_acc: 1.0000
Epoch 894 train_loss: 0.0106 train_acc: 1.0000
Epoch 895 train_loss: 0.0192 train_acc: 1.0000
Epoch 896 train_loss: 0.0228 train_acc: 0.9929
Epoch 897 train_loss: 0.0447 train_acc: 0.9786
Epoch 898 train_loss: 0.0141 train_acc: 1.0000
Epoch 899 train_loss: 0.0232 train_acc: 1.0000
Accuracy: 0.8210
Epoch 900 train_loss: 0.0237 train_acc: 0.9929
Epoch 901 train_loss: 0.0172 train_acc: 1.0000
Epoch 902 train_loss: 0.0154 train_acc: 1.0000
Epoch 903 train_loss: 0.0204 train_acc: 1.0000
Epoch 904 train_loss: 0.01

In [42]:
out = model(data)
loss = criterion(out[test_mask], data.y[test_mask])
_, pred = torch.max(out[test_mask], dim=1)
correct = (pred == data.y[test_mask]).sum().item()
acc = correct/len(test_mask)
print("test_loss: {:.4f} test_acc: {:.4f}".format(loss.item(), acc))

test_loss: 0.8173 test_acc: 0.7940


# GNN Model

In [2]:
from torch_geometric.datasets import Planetoid
import torch

In [3]:
dataset_cora = Planetoid(root='./cora/', name='Cora')
# dataset = Planetoid(root='./citeseer',name='Citeseer')
# dataset = Planetoid(root='./pubmed/',name='Pubmed')
print(dataset_cora)

Cora()


In [ ]:
# from torch_geometric.datasets import Reddit
# dataset_reddit = Reddit(root='./reddit/')
# print(dataset_reddit)

In [ ]:
dataset = dataset_cora

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv

In [ ]:
# gpu_data = list(nvsmi.get_gpus())[0]
# start_gpu_util = gpu_data.gpu_util
# start_gpu_mem_use = gpu_data.mem_used
# total_gpu_mem = gpu_data.mem_total
disk_io_counter = psutil.disk_io_counters()
disk_total = disk_io_counter[2] + disk_io_counter[3]  # read_bytes + write_bytes
p = psutil.Process()
group_mem_rss = []
# group_gpu_util = []
# group_gpu_mem_use = []
group_disk_usage = []
group_iowait = []
t_status = True

def get_gpu_info():
#     gpu_data = list(nvsmi.get_gpus())[0]
    while True:
        if not t_status:
            break
#         group_gpu_util.append(gpu_data.gpu_util)
#         group_gpu_mem_use.append(gpu_data.mem_used)
        group_mem_rss.append(psutil.Process(os.getpid()).memory_info().rss)
        io_counters = p.io_counters()
        disk_usage = io_counters[2] + io_counters[3]
        group_disk_usage.append(disk_usage)  # read_bytes + write_bytes
        try:
            a = psutil.cpu_times_percent().iowait
        except:
            a = 0
        group_iowait.append(a)
#         print(f"{disk_usage / 1024 / 1024} Mb/s")
#         time.sleep(0.09)

In [ ]:
path = "/mnt/NVme/project_moka/pubmed/"
path = "/mnt/ramfs/project_moka/pubmed/"
#path = "/mnt/ext4ramdisk/project_moka/pubmed/"
#path = "./pubmed/"

path_Cora = "/mnt/NVme/project_moka/data/Cora/"
# path_Cora = "/mnt/ramfs/project_moka/data/Cora/"
#path_Cora = "/mnt/ext4ramdisk/project_moka/data/Cora/"
#path_Cora = "./data/Cora/"

path_pm = "/mnt/NVme/datasets/"
path_ram = "/mnt/ramfs/datasets/"

times = 15
total_time = 0
total_run_time = 0
batch_size = 128
epoch_num = 20

# pre-load Planetoid
t = threading.Thread(target=get_gpu_info)
t.start()
start_time = time.time()
dataset_test = Planetoid(root="./cora/", name='Cora')
end_time = time.time()
init_sample_time = end_time - start_time
print('Dataloading time:{}'.format(end_time - start_time))
t_status = False
t.join()
# print(f"GPU 显存占用: {np.mean(group_gpu_mem_use)}Mb")
# print(f"GPU 显存占用率: {np.mean(group_gpu_mem_use) * 100 / total_gpu_mem}%")
# print(f"GPU 平均使用率: {np.mean(group_gpu_util) - start_gpu_util}%")
# tmp_ = sum(np.where(np.array(group_gpu_util) - start_gpu_util > 1, True, False))
# print(f"GPU 空闲率: {(len(group_gpu_util) - tmp_) * 100 / len(group_gpu_util)}%")
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')
#dataset_test = Planetoid(root=path_Cora, name='Cora')
# the dataset for test is shown below (different from the above)
# dataset_test = Planetoid(root='./data/Cora/', name='Cora')


dataset = dataset_test


data = dataset[0]  # Get the first graph object.


In [ ]:
from torch_geometric.data import NeighborSampler
start_time = time.time()
t = threading.Thread(target=get_gpu_info)
t.start()
train_loader = NeighborSampler(data.edge_index, node_idx=data.train_mask,
                               sizes=[10, 10], batch_size=16, shuffle=True,
                               num_workers=12)
t_status = False
t.join()
end_time = time.time()
init_sample_time = end_time - start_time
print('NeighborSampler time:{}'.format(end_time - start_time))
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')

In [ ]:
start_time = time.time()
t = threading.Thread(target=get_gpu_info)
t.start()
subgraph_loader = NeighborSampler(dataset[0].edge_index, node_idx=None, sizes=[-1],
                                  batch_size=1024, shuffle=False,
                                  num_workers=12)
t_status = False
t.join()
end_time = time.time()
init_sample_time = end_time - start_time
print('NeighborSampler time:{}'.format(end_time - start_time))
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')

In [ ]:
from torch_geometric.data import ClusterData, ClusterLoader, DataLoader

torch.manual_seed(1234)
cluster_data = ClusterData(data, num_parts=128)  # 1. Create subgraphs.
train_loader = ClusterLoader(cluster_data, batch_size=batch_size,
                             shuffle=True)  # 2. Stochastic partitioning scheme.

# Build GNN Network

In [ ]:
class SAGENet(nn.Module):
    def __init__(self):
        super(SAGENet, self).__init__()
        self.conv1 = SAGEConv(dataset.num_node_features, 128)
        self.conv2 = SAGEConv(128, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

# With sampling

In [ ]:
class SAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SAGENet, self).__init__()

        self.num_layers = 2

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, adjs):
        # `train_loader` computes the k-hop neighborhood of a batch of nodes,
        # and returns, for each layer, a bipartite graph object, holding the
        # bipartite edges `edge_index`, the index `e_id` of the original edges,
        # and the size/shape `size` of the bipartite graph.
        # Target nodes are also included in the source nodes so that one can
        # easily apply skip-connections or add self-loops.

        lin_times = 0
        mes_times = 0
        aggr_times = 0
        up_times = 0

        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x, linear_time, message_time, aggregate_time, update_time = self.convs[i]((x, x_target), edge_index)
            lin_times += linear_time
            mes_times += message_time
            aggr_times += aggregate_time
            up_times += update_time
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        return x.log_softmax(dim=-1), lin_times, mes_times, aggr_times, up_times

    def inference(self, x_all):
        # pbar = tqdm(total=x_all.size(0) * self.num_layers)
        # pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x, linear_time, message_time, aggregate_time, update_time = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x.cpu())

                # pbar.update(batch_size)

            x_all = torch.cat(xs, dim=0)

        # pbar.close()

        return x_all

In [ ]:
class GATNet(torch.nn.Module):
    def __init__(self):
        super(GATNet, self).__init__()
        self.gat1 = GATConv(dataset.num_node_features, 8, 8, dropout=0.6)
        self.gat2 = GATConv(64, 7, 1, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gat1(x, edge_index)
        x = self.gat2(x, edge_index)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
class GCNNet(torch.nn.Module):
    def __init__(self):
        super(GCNNet, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.log_softmax(x, dim=1)
        
        return x


In [ ]:
model = SAGENet(dataset.num_features, 256, dataset.num_classes)
print(model)

In [ ]:
model = GATNet()
print(model)

In [ ]:
model = GCNNet()
print(model)

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset_cora[0].to(device)
print(data)

In [ ]:
x = data.x.to(device)
y = data.y.squeeze().to(device)

In [ ]:
import torch.optim as optim

In [ ]:
# criterion = nn.CrossEntropyLoss().to(device)
criterion = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Test system evaluation index

In [ ]:
# gpu_data = list(nvsmi.get_gpus())[0]
# start_gpu_util = gpu_data.gpu_util
# start_gpu_mem_use = gpu_data.mem_used
# total_gpu_mem = gpu_data.mem_total
disk_io_counter = psutil.disk_io_counters()
disk_total = disk_io_counter[2] + disk_io_counter[3]  # read_bytes + write_bytes
p = psutil.Process()
group_mem_rss = []
# group_gpu_util = []
# group_gpu_mem_use = []
group_disk_usage = []
group_iowait = []
t_status = True

In [ ]:
def get_gpu_info():
#     gpu_data = list(nvsmi.get_gpus())[0]
    while True:
        if not t_status:
            break
#         group_gpu_util.append(gpu_data.gpu_util)
#         group_gpu_mem_use.append(gpu_data.mem_used)
        group_mem_rss.append(psutil.Process(os.getpid()).memory_info().rss)
        io_counters = p.io_counters()
        disk_usage = io_counters[2] + io_counters[3]
        group_disk_usage.append(disk_usage)  # read_bytes + write_bytes
        try:
            a = psutil.cpu_times_percent().iowait
        except:
            a = 0
        group_iowait.append(a)
#         print(f"{disk_usage / 1024 / 1024} Mb/s")

# Training

In [ ]:
def train(epoch):
    model.train()

    # pbar = tqdm(total=int(data.train_mask.sum()))
    # pbar.set_description(f'Epoch {epoch:02d}')

    total_lin_time = 0
    total_mes_time = 0
    total_aggr_time = 0
    total_up_time = 0

    total_sample_time = 0

    total_loss = total_correct = 0
    start_time = time.time()
    
    for batch_size, n_id, adjs in train_loader:
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        end_time = time.time()
        total_sample_time += (end_time - start_time)

        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out, lin_time, mes_time, aggr_time, up_time = model(x[n_id], adjs)

        total_lin_time += lin_time
        total_mes_time += mes_time
        total_aggr_time += aggr_time
        total_up_time += up_time

        loss = F.nll_loss(out, y[n_id[:batch_size]])
#         loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        # pbar.update(batch_size)
        start_time = time.time()

    # pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / int(data.train_mask.sum())

    return loss, approx_acc, total_lin_time, total_mes_time, total_aggr_time, total_up_time, total_sample_time


In [ ]:
@torch.no_grad()
def test():
    model.eval()

    out = model.inference(x)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    results = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]

    return results

In [ ]:
lin_times = []
mes_times = []
aggr_times = []
up_times = []
sample_times = []
for epoch in range(1, 11):
    if epoch == 1:
        t = threading.Thread(target=get_gpu_info)
        t.start()
    loss, acc, lin_time, mes_time, aggr_time, up_time, sample_time = train(epoch)

    lin_times.append(lin_time)
    mes_times.append(mes_time)
    aggr_times.append(aggr_time)
    up_times.append(up_time)
    sample_times.append(sample_time)

    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')

    train_acc, val_acc, test_acc = test()
    print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')
t_status = False
t.join()
print("Average linear time:", 1000 * np.mean(lin_times), 'ms')
print("Average message time:", 1000 * np.mean(mes_times), 'ms')
print("Average aggregate time:", 1000 * np.mean(aggr_times), 'ms')
print("Average update time:", 1000 * np.mean(up_times), 'ms')
print("Average sample time:", (1000 * np.mean(sample_times) + init_sample_time), 'ms')

In [ ]:
# model.train()
# total_lin_time = 0
# total_mes_time = 0
# total_aggr_time = 0
# total_up_time = 0

# total_sample_time = 0

# start_time = time.time()
# for epoch in range(200):
#     if epoch == 1:
#         t = threading.Thread(target=get_gpu_info)
#         t.start()
#     out = model(data)
#     loss = criterion(out[data.train_mask], data.y[data.train_mask])

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     _, pred = torch.max(out[data.train_mask], dim=1)
#     correct = (pred == data.y[data.train_mask]).sum().item()
#     print(correct)
#     acc = correct/data.train_mask.sum().item()
#     print(data.train_mask.sum())

#     print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
#         epoch, loss.item(), acc))
# t_status = False
# t.join()

In [ ]:
# print(f"GPU 显存占用: {np.mean(group_gpu_mem_use)}Mb")
# print(f"GPU 显存占用率: {np.mean(group_gpu_mem_use) * 100 / total_gpu_mem}%")
# print(f"GPU 平均使用率: {np.mean(group_gpu_util) - start_gpu_util}%")
# tmp_ = sum(np.where(np.array(group_gpu_util) - start_gpu_util > 1, True, False))
# print(f"GPU 空闲率: {(len(group_gpu_util) - tmp_) * 100 / len(group_gpu_util)}%")
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')

In [ ]:
out = model(data)
loss = criterion(out[data.test_mask], data.y[data.test_mask])
_, pred = torch.max(out[data.test_mask], dim=1)
correct = (pred == data.y[data.test_mask]).sum().item()
acc = correct/data.test_mask.sum().item()
print("test_loss: {:.4f} test_acc: {:.4f}".format(loss.item(), acc))

In [ ]:
# import os

# f = os.open('file.txt', os.O_CREAT | os.O_TRUNC | os.O_DIRECT | os.O_RDWR)
# s = ' ' * 1024
# os.write(f, s.encode())


In [ ]:
import nvsmi
import threading
import psutil
import os
import os.path as osp

disk_io_counter = psutil.disk_io_counters()
disk_total = disk_io_counter[2] + disk_io_counter[3]  # read_bytes + write_bytes
p = psutil.Process()
group_mem_rss = []
group_disk_usage = []
group_iowait = []
t_status = True

def get_gpu_info():
    while True:
        if not t_status:
            break
        group_mem_rss.append(psutil.Process(os.getpid()).memory_info().rss)
        io_counters = p.io_counters()
        disk_usage = io_counters[2] + io_counters[3]
        group_disk_usage.append(disk_usage)  # read_bytes + write_bytes
        try:
            a = psutil.cpu_times_percent().iowait
        except:
            a = 0
        group_iowait.append(a)
#         print(f"{disk_usage / 1024 / 1024} Mb/s")
#         time.sleep(0.09)

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import Reddit
import torch
from torch_geometric.data import NeighborSampler
# from NS import NeighborSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from SAGEConv import SAGEConv
from tqdm import tqdm
import torch.optim as optim
import numpy as np
import time

# from torch.utils.data import DataLoader

dataset = Planetoid(root='./cora/', name='Cora')
# dataset = Planetoid(root='./citeseer',name='Citeseer')
# dataset = Planetoid(root='./pubmed/',name='Pubmed')
# dataset = Reddit(root='./reddit/')
print(dataset)

start_time = time.time()
t = threading.Thread(target=get_gpu_info)
t.start()
train_loader = NeighborSampler(dataset[0].edge_index, node_idx=dataset[0].train_mask,
                               sizes=[10, 10], batch_size=16, shuffle=True,
                               num_workers=6)
t_status = False
t.join()

# Reddit
# train_loader = NeighborSampler(dataset[0].edge_index, node_idx=dataset[0].train_mask,
#                                sizes=[25, 10], batch_size=1024, shuffle=True,
#                                num_workers=12)

end_time = time.time()
init_sample_time = end_time - start_time
print('NeighborSampler time:{}'.format(end_time - start_time))
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')


subgraph_loader = NeighborSampler(dataset[0].edge_index, node_idx=None, sizes=[-1],
                                  batch_size=16, shuffle=False,
                                  num_workers=6)

In [ ]:
gpu_data = list(nvsmi.get_gpus())[0]
start_gpu_util = gpu_data.gpu_util
start_gpu_mem_use = gpu_data.mem_used
total_gpu_mem = gpu_data.mem_total
disk_io_counter = psutil.disk_io_counters()
disk_total = disk_io_counter[2] + disk_io_counter[3]  # read_bytes + write_bytes
p = psutil.Process()
group_mem_rss = []
group_gpu_util = []
group_gpu_mem_use = []
group_disk_usage = []
group_iowait = []
t_status = True

def get_gpu_info():
    gpu_data = list(nvsmi.get_gpus())[0]
    while True:
        if not t_status:
            break
        group_gpu_util.append(gpu_data.gpu_util)
        group_gpu_mem_use.append(gpu_data.mem_used)
        group_mem_rss.append(psutil.Process(os.getpid()).memory_info().rss)
        io_counters = p.io_counters()
        disk_usage = io_counters[2] + io_counters[3]
        group_disk_usage.append(disk_usage)  # read_bytes + write_bytes
        try:
            a = psutil.cpu_times_percent().iowait
        except:
            a = 0
        group_iowait.append(a)


class SAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SAGENet, self).__init__()

        self.num_layers = 2

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, adjs):
        # `train_loader` computes the k-hop neighborhood of a batch of nodes,
        # and returns, for each layer, a bipartite graph object, holding the
        # bipartite edges `edge_index`, the index `e_id` of the original edges,
        # and the size/shape `size` of the bipartite graph.
        # Target nodes are also included in the source nodes so that one can
        # easily apply skip-connections or add self-loops.

        lin_times = 0
        mes_times = 0
        aggr_times = 0
        up_times = 0

        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x, linear_time, message_time, aggregate_time, update_time = self.convs[i]((x, x_target), edge_index)
            lin_times += linear_time
            mes_times += message_time
            aggr_times += aggregate_time
            up_times += update_time
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        return x.log_softmax(dim=-1), lin_times, mes_times, aggr_times, up_times

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x, linear_time, message_time, aggregate_time, update_time = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x.cpu())

                pbar.update(batch_size)

            x_all = torch.cat(xs, dim=0)

        pbar.close()

        return x_all


# cora
model = SAGENet(dataset.num_features, 16, dataset.num_classes)

# Reddit
# model = SAGENet(dataset.num_features, 256, dataset.num_classes)
print(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset[0].to(device)
print(data)

x = data.x.to(device)
y = data.y.squeeze().to(device)

criterion = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


def train(epoch):
    model.train()

    pbar = tqdm(total=int(data.train_mask.sum()))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_lin_time = 0
    total_mes_time = 0
    total_aggr_time = 0
    total_up_time = 0

    total_sample_time = 0

    total_loss = total_correct = 0
    start_time = time.time()
    for batch_size, n_id, adjs in train_loader:
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        end_time = time.time()
        total_sample_time += (end_time - start_time)

        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out, lin_time, mes_time, aggr_time, up_time = model(x[n_id], adjs)

        total_lin_time += lin_time
        total_mes_time += mes_time
        total_aggr_time += aggr_time
        total_up_time += up_time

        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        pbar.update(batch_size)
        start_time = time.time()

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / int(data.train_mask.sum())

    return loss, approx_acc, total_lin_time, total_mes_time, total_aggr_time, total_up_time, total_sample_time


@torch.no_grad()
def test():
    model.eval()

    out = model.inference(x)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    results = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]

    return results


lin_times = []
mes_times = []
aggr_times = []
up_times = []
sample_times = []
for epoch in range(1, 11):
    if epoch == 1:
        t = threading.Thread(target=get_gpu_info)
        t.start()
    loss, acc, lin_time, mes_time, aggr_time, up_time, sample_time = train(epoch)

    lin_times.append(lin_time)
    mes_times.append(mes_time)
    aggr_times.append(aggr_time)
    up_times.append(up_time)
    sample_times.append(sample_time)

    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')

    train_acc, val_acc, test_acc = test()
    print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')
t_status = False
t.join()
print("Average linear time:", 1000 * np.mean(lin_times), 'ms')
print("Average message time:", 1000 * np.mean(mes_times), 'ms')
print("Average aggregate time:", 1000 * np.mean(aggr_times), 'ms')
print("Average update time:", 1000 * np.mean(up_times), 'ms')
print("Average sample time:", (1000 * np.mean(sample_times) + init_sample_time), 'ms')
print("========================================================")
print(f"GPU 显存占用: {np.mean(group_gpu_mem_use)}Mb")
print(f"GPU 显存占用率: {np.mean(group_gpu_mem_use) * 100 / total_gpu_mem}%")
print(f"GPU 平均使用率: {np.mean(group_gpu_util) - start_gpu_util}%")
tmp_ = sum(np.where(np.array(group_gpu_util) - start_gpu_util > 1, True, False))
print(f"GPU 空闲率: {(len(group_gpu_util) - tmp_) * 100 / len(group_gpu_util)}%")
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')

In [ ]:
import nvsmi
print(list(nvsmi.get_gpus())[0].gpu_util)

In [24]:
# model.train()
# total_lin_time = 0
# total_mes_time = 0
# total_aggr_time = 0
# total_up_time = 0

# total_sample_time = 0

# start_time = time.time()
# for epoch in range(200):
#     if epoch == 1:
#         t = threading.Thread(target=get_gpu_info)
#         t.start()
#     out = model(data)
#     loss = criterion(out[data.train_mask], data.y[data.train_mask])

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     _, pred = torch.max(out[data.train_mask], dim=1)
#     correct = (pred == data.y[data.train_mask]).sum().item()
#     print(correct)
#     acc = correct/data.train_mask.sum().item()
#     print(data.train_mask.sum())

#     print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
#         epoch, loss.item(), acc))
# t_status = False
# t.join()

In [34]:
# print(f"GPU 显存占用: {np.mean(group_gpu_mem_use)}Mb")
# print(f"GPU 显存占用率: {np.mean(group_gpu_mem_use) * 100 / total_gpu_mem}%")
# print(f"GPU 平均使用率: {np.mean(group_gpu_util) - start_gpu_util}%")
# tmp_ = sum(np.where(np.array(group_gpu_util) - start_gpu_util > 1, True, False))
# print(f"GPU 空闲率: {(len(group_gpu_util) - tmp_) * 100 / len(group_gpu_util)}%")
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')

内存使用：0.4485 GB
磁盘IO使用：0.2824 GB/s
磁盘IO使用率：12.1613%
cpu iowait：0.0


In [22]:
out = model(data)
loss = criterion(out[data.test_mask], data.y[data.test_mask])
_, pred = torch.max(out[data.test_mask], dim=1)
correct = (pred == data.y[data.test_mask]).sum().item()
acc = correct/data.test_mask.sum().item()
print("test_loss: {:.4f} test_acc: {:.4f}".format(loss.item(), acc))

test_loss: 0.7891 test_acc: 0.7790


In [8]:
# import os

# f = os.open('file.txt', os.O_CREAT | os.O_TRUNC | os.O_DIRECT | os.O_RDWR)
# s = ' ' * 1024
# os.write(f, s.encode())


In [2]:
import nvsmi
import threading
import psutil
import os
import os.path as osp

disk_io_counter = psutil.disk_io_counters()
disk_total = disk_io_counter[2] + disk_io_counter[3]  # read_bytes + write_bytes
p = psutil.Process()
group_mem_rss = []
group_disk_usage = []
group_iowait = []
t_status = True

def get_gpu_info():
    while True:
        if not t_status:
            break
        group_mem_rss.append(psutil.Process(os.getpid()).memory_info().rss)
        io_counters = p.io_counters()
        disk_usage = io_counters[2] + io_counters[3]
        group_disk_usage.append(disk_usage)  # read_bytes + write_bytes
        try:
            a = psutil.cpu_times_percent().iowait
        except:
            a = 0
        group_iowait.append(a)
#         print(f"{disk_usage / 1024 / 1024} Mb/s")
#         time.sleep(0.09)

In [3]:
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import Reddit
import torch
from torch_geometric.data import NeighborSampler
# from NS import NeighborSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from SAGEConv import SAGEConv
from tqdm import tqdm
import torch.optim as optim
import numpy as np
import time

# from torch.utils.data import DataLoader

dataset = Planetoid(root='./cora/', name='Cora')
# dataset = Planetoid(root='./citeseer',name='Citeseer')
# dataset = Planetoid(root='./pubmed/',name='Pubmed')
# dataset = Reddit(root='./reddit/')
print(dataset)

start_time = time.time()
t = threading.Thread(target=get_gpu_info)
t.start()
train_loader = NeighborSampler(dataset[0].edge_index, node_idx=dataset[0].train_mask,
                               sizes=[10, 10], batch_size=16, shuffle=True,
                               num_workers=6)
t_status = False
t.join()

# Reddit
# train_loader = NeighborSampler(dataset[0].edge_index, node_idx=dataset[0].train_mask,
#                                sizes=[25, 10], batch_size=1024, shuffle=True,
#                                num_workers=12)

end_time = time.time()
init_sample_time = end_time - start_time
print('NeighborSampler time:{}'.format(end_time - start_time))
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')


subgraph_loader = NeighborSampler(dataset[0].edge_index, node_idx=None, sizes=[-1],
                                  batch_size=16, shuffle=False,
                                  num_workers=6)

Cora()


D:\AnacondaSystem\envs\anlp_cw2\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.NeighborSampler' is deprecated, use 'loader.NeighborSampler' instead
  warnings.warn(out)


NeighborSampler time:0.5884826183319092
内存使用：0.3019 GB
磁盘IO使用：0.0421 GB/s
磁盘IO使用率：0.2337%
cpu iowait：0.0


In [ ]:
gpu_data = list(nvsmi.get_gpus())[0]
start_gpu_util = gpu_data.gpu_util
start_gpu_mem_use = gpu_data.mem_used
total_gpu_mem = gpu_data.mem_total
disk_io_counter = psutil.disk_io_counters()
disk_total = disk_io_counter[2] + disk_io_counter[3]  # read_bytes + write_bytes
p = psutil.Process()
group_mem_rss = []
group_gpu_util = []
group_gpu_mem_use = []
group_disk_usage = []
group_iowait = []
t_status = True

def get_gpu_info():
    gpu_data = list(nvsmi.get_gpus())[0]
    while True:
        if not t_status:
            break
        group_gpu_util.append(gpu_data.gpu_util)
        group_gpu_mem_use.append(gpu_data.mem_used)
        group_mem_rss.append(psutil.Process(os.getpid()).memory_info().rss)
        io_counters = p.io_counters()
        disk_usage = io_counters[2] + io_counters[3]
        group_disk_usage.append(disk_usage)  # read_bytes + write_bytes
        try:
            a = psutil.cpu_times_percent().iowait
        except:
            a = 0
        group_iowait.append(a)


class SAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SAGENet, self).__init__()

        self.num_layers = 2

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, adjs):
        # `train_loader` computes the k-hop neighborhood of a batch of nodes,
        # and returns, for each layer, a bipartite graph object, holding the
        # bipartite edges `edge_index`, the index `e_id` of the original edges,
        # and the size/shape `size` of the bipartite graph.
        # Target nodes are also included in the source nodes so that one can
        # easily apply skip-connections or add self-loops.

        lin_times = 0
        mes_times = 0
        aggr_times = 0
        up_times = 0

        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x, linear_time, message_time, aggregate_time, update_time = self.convs[i]((x, x_target), edge_index)
            lin_times += linear_time
            mes_times += message_time
            aggr_times += aggregate_time
            up_times += update_time
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        return x.log_softmax(dim=-1), lin_times, mes_times, aggr_times, up_times

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x, linear_time, message_time, aggregate_time, update_time = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x.cpu())

                pbar.update(batch_size)

            x_all = torch.cat(xs, dim=0)

        pbar.close()

        return x_all


# cora
model = SAGENet(dataset.num_features, 16, dataset.num_classes)

# Reddit
# model = SAGENet(dataset.num_features, 256, dataset.num_classes)
print(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset[0].to(device)
print(data)

x = data.x.to(device)
y = data.y.squeeze().to(device)

criterion = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


def train(epoch):
    model.train()

    pbar = tqdm(total=int(data.train_mask.sum()))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_lin_time = 0
    total_mes_time = 0
    total_aggr_time = 0
    total_up_time = 0

    total_sample_time = 0

    total_loss = total_correct = 0
    start_time = time.time()
    for batch_size, n_id, adjs in train_loader:
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        end_time = time.time()
        total_sample_time += (end_time - start_time)

        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out, lin_time, mes_time, aggr_time, up_time = model(x[n_id], adjs)

        total_lin_time += lin_time
        total_mes_time += mes_time
        total_aggr_time += aggr_time
        total_up_time += up_time

        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        pbar.update(batch_size)
        start_time = time.time()

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / int(data.train_mask.sum())

    return loss, approx_acc, total_lin_time, total_mes_time, total_aggr_time, total_up_time, total_sample_time


@torch.no_grad()
def test():
    model.eval()

    out = model.inference(x)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    results = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]

    return results


lin_times = []
mes_times = []
aggr_times = []
up_times = []
sample_times = []
for epoch in range(1, 11):
    if epoch == 1:
        t = threading.Thread(target=get_gpu_info)
        t.start()
    loss, acc, lin_time, mes_time, aggr_time, up_time, sample_time = train(epoch)

    lin_times.append(lin_time)
    mes_times.append(mes_time)
    aggr_times.append(aggr_time)
    up_times.append(up_time)
    sample_times.append(sample_time)

    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')

    train_acc, val_acc, test_acc = test()
    print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')
t_status = False
t.join()
print("Average linear time:", 1000 * np.mean(lin_times), 'ms')
print("Average message time:", 1000 * np.mean(mes_times), 'ms')
print("Average aggregate time:", 1000 * np.mean(aggr_times), 'ms')
print("Average update time:", 1000 * np.mean(up_times), 'ms')
print("Average sample time:", (1000 * np.mean(sample_times) + init_sample_time), 'ms')
print("========================================================")
print(f"GPU 显存占用: {np.mean(group_gpu_mem_use)}Mb")
print(f"GPU 显存占用率: {np.mean(group_gpu_mem_use) * 100 / total_gpu_mem}%")
print(f"GPU 平均使用率: {np.mean(group_gpu_util) - start_gpu_util}%")
tmp_ = sum(np.where(np.array(group_gpu_util) - start_gpu_util > 1, True, False))
print(f"GPU 空闲率: {(len(group_gpu_util) - tmp_) * 100 / len(group_gpu_util)}%")
print(f'内存使用：{np.mean(group_mem_rss) / 1024 / 1024 / 1024:.4f} GB')
print(f'磁盘IO使用：{np.mean(group_disk_usage) / 1024 / 1024 / 1024:.4f} GB/s')
print(f'磁盘IO使用率：{np.mean(group_disk_usage) * 100 / disk_total:.4f}%')
print(f'cpu iowait：{np.mean(group_iowait)}')

SAGENet(
  (convs): ModuleList(
    (0): SAGEConv(1433, 16)
    (1): SAGEConv(16, 7)
  )
)
cuda
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Epoch 01: 100%|██████████████████████████████████████████████████████████████████████| 140/140 [00:23<00:00,  5.88it/s]


Epoch 01, Loss: 1.8575, Approx. Train: 0.1786


Evaluating: 100%|██████████████████████████████████████████████████████████████████| 5416/5416 [03:01<00:00, 29.76it/s]


Train: 0.5143, Val: 0.2540, Test: 0.2400


...

Evaluating: 100%|██████████████████████████████████████████████████████████████████| 5416/5416 [02:55<00:00, 30.90it/s]


Train: 1.0000, Val: 0.7600, Test: 0.7910


Epoch 09: 100%|██████████████████████████████████████████████████████████████████████| 140/140 [00:24<00:00,  5.64it/s]


Epoch 09, Loss: 0.0715, Approx. Train: 0.9786


Evaluating:  64%|██████████████████████████████████████████▎                       | 3476/5416 [02:05<01:10, 27.64it/s]

In [21]:
import nvsmi
print(list(nvsmi.get_gpus())[0].gpu_util)

30.0
